In [1]:
pip install tensorflow

In [2]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)
 
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import LSTM, Dropout
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Dense, Activation, Dropout, RepeatVector
from tensorflow.keras.optimizers import  RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

In [3]:
text = open('1661-0.txt',encoding = "utf8").read().lower()
print('corpus length:', len(text))

corpus length: 581888


In [4]:
character = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(character))
indices_char = dict((i, c) for i, c in enumerate(character))
 
print(f'unique chars: {len(character)}')

unique chars: 73


In [5]:
seq_len = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - seq_len, step):
    sentences.append(text[i: i + seq_len ])
    next_chars.append(text[i + seq_len])
print(f'num training examples: {len(sentences)}')

num training examples: 193950


In [6]:
X = np.zeros((len(sentences), seq_len, len(character)), dtype=np.bool)
y = np.zeros((len(sentences), len(character)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

sentences[124]
next_chars[100]

C:\Users\Dell\AppData\Local\Temp\ipykernel_6376\1029853919.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((len(sentences), seq_len, len(character)), dtype=np.bool)
C:\Users\Dell\AppData\Local\Temp\ipykernel_6376\1029853919.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(character)), dtype=np.b

'e'

In [7]:
X[0][0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True])

In [8]:
X.shape

(193950, 40, 73)

In [9]:
y.shape

(193950, 73)

In [12]:
model = Sequential()
model.add(LSTM(128, input_shape=(seq_len, len(character))))
model.add(Dense(len(character)))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 128)               103424    
                                                                 
 dense_2 (Dense)             (None, 73)                9417      
                                                                 
 activation_2 (Activation)   (None, 73)                0         
                                                                 
Total params: 112841 (440.79 KB)
Trainable params: 112841 (440.79 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, y,
                    validation_split=0.01, 
                    batch_size=128,
                    epochs=7, 
                    shuffle=True).history

Epoch 1/7
1501/1501 [==============================] - 217s 141ms/step - loss: 2.0617 - accuracy: 0.3972 - val_loss: 2.2911 - val_accuracy: 0.3557
Epoch 2/7
1501/1501 [==============================] - 200s 134ms/step - loss: 1.6740 - accuracy: 0.4976 - val_loss: 2.0718 - val_accuracy: 0.4263
Epoch 3/7
1501/1501 [==============================] - 199s 133ms/step - loss: 1.5511 - accuracy: 0.5317 - val_loss: 1.9878 - val_accuracy: 0.4505
Epoch 4/7
1501/1501 [==============================] - 200s 133ms/step - loss: 1.4870 - accuracy: 0.5496 - val_loss: 1.9707 - val_accuracy: 0.4552
Epoch 5/7
1501/1501 [==============================] - 205s 137ms/step - loss: 1.4437 - accuracy: 0.5607 - val_loss: 1.9351 - val_accuracy: 0.4701
Epoch 6/7
1501/1501 [==============================] - 226s 150ms/step - loss: 1.4143 - accuracy: 0.5689 - val_loss: 1.9418 - val_accuracy: 0.4716
Epoch 7/7
1501/1501 [==============================] - 193s 128ms/step - loss: 1.3930 - accuracy: 0.5753 - val_loss: 1

In [20]:
model.save('NW_keras_model.h5')
pickle.dump(history, open("history.p", "wb"))

In [22]:
def prepare_input(text):
    x = np.zeros((1, seq_len, len(character)))
    for t, char in enumerate(text):
        x[0, t, char_indices[char]] = 1.
        
    return x

prepare_input("This is an example of input of our model".lower())

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [26]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    
    return heapq.nlargest(top_n, range(len(preds)), preds.take)
def predict_completion(text):
    original_text = text
    generated = text
    completion = ''
    while True:
        x = prepare_input(text)
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, top_n=1)[0]
        next_char = indices_char[next_index]
        text = text[1:] + next_char
        completion += next_char
        if len(original_text + completion) + 2 > len(original_text) and next_char == ' ':
            return completion
def predict_completions(text, n=3):
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [indices_char[idx] + predict_completion(text[1:] + indices_char[idx]) for idx in next_indices]

In [28]:
quotes = [   
    "Deep learning is subset of machine learning,which is essentially a neural network with three or more layers.",
    "Machine learning  is the study of computer algorithms that improve automatically through experience and by the use of data.",
    "It is not a lack of love, but a lack of friendship that makes unhappy marriages.",
    "Recurrent neural network (RNN) is a type of artificial neural network which uses sequential data or time series data.",
    "Thanks for spending your precious time to view my work."  ]

In [29]:
for i in quotes:
    seq = i[:40].lower()
    print(seq)
    print(predict_completions(seq, 7))
    print()

deep learning is subset of machine learn
['ed ', ' of ', '. ', ', ', '\nto ', 'ing ', 'ce ']

machine learning  is the study of comput
['e ', 'ion. ', ' of ', 'ches ', 'ual ', 'h ', 'ain ']

it is not a lack of love, but a lack of 
['the ', 'a ', 'sherlock ', 'his ', 'my ', 'it ', 'course ']

recurrent neural network (rnn) is a type
['writing ', 'long ', 'ters ', ' of ', 'ssing ', 'ce ', 'ision ']

thanks for spending your precious time t
['o ', 'hat ', 'ermed ', 'ake ', 'rain ', 'ime ', 'wo ']

